In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Red_Hair"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Red_Hair/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Red_Hair/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Red_Hair/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Red_Hair/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Search for directories related to Red Hair
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Red hair is associated with melanoma risk, so look for melanoma or skin cancer datasets
matching_dirs = [dir_name for dir_name in tcga_dirs 
                if any(term in dir_name.lower() for term in 
                       ["melanoma", "skin cancer", "skin", "skcm"])]

if not matching_dirs:
    print(f"No matching directory found for trait: {trait}")
    
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    print(f"Task marked as completed. {trait} is not directly represented in the TCGA dataset.")
else:
    # If we found matching directories
    print(f"Found matching directories: {matching_dirs}")
    
    # Select the most specific directory for melanoma (which may have red hair data)
    if "TCGA_Melanoma_(SKCM)" in matching_dirs:
        selected_dir = "TCGA_Melanoma_(SKCM)"  # Choose the most specific match
    else:
        selected_dir = matching_dirs[0]  # Default to first match if specific one not found
    
    print(f"Selected directory: {selected_dir}")
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Found matching directories: ['TCGA_Melanoma_(SKCM)', 'TCGA_Ocular_melanomas_(UVM)']
Selected directory: TCGA_Melanoma_(SKCM)


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'breslow_depth_value', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_submitted_specimen_dx', 'distant_metastasis_anatomic_site', 'followup_case_report_form_submission_reason', 'form_completion_date', 'gender', 'height', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consent_verified', 'initial_weight', 'interferon_90_day_prior_excision_admin_indicator', 'is_ffpe', 'lactate_dehydrogenase_result', 'lost_follow_up', 'malignant_neoplasm_mitotic_count_rate', 'melanoma_clark_level_value'

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify candidate demographic columns
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['gender']

# 2. Load the clinical data to preview
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Melanoma_(SKCM)')
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract the candidate columns for preview
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [71.0, 82.0, 82.0, 46.0, 74.0], 'days_to_birth': [-26176.0, -30286.0, -30163.0, -17025.0, -27124.0], 'year_of_initial_pathologic_diagnosis': [2012.0, 2009.0, 2013.0, 2010.0, 2010.0]}

Gender columns preview:
{'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Analyze the available demographic information

# Check age columns
# We have 3 potential age columns:
# 1. age_at_initial_pathologic_diagnosis - direct age values
# 2. days_to_birth - negative days (can be converted to years by dividing by -365)
# 3. year_of_initial_pathologic_diagnosis - not useful for age calculation without additional info

# Check gender columns
# We have only one gender column: 'gender'

# Select the most appropriate columns
age_col = 'age_at_initial_pathologic_diagnosis'  # Direct age values are most useful
gender_col = 'gender'  # Only option available

# Print chosen columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Use the Melanoma directory identified in Step 1
selected_dir = "TCGA_Melanoma_(SKCM)"
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for melanoma patients from TCGA, which is relevant for studying Red_Hair trait due to its association with melanoma risk."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 481 samples


After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (474, 19851) (samples x features)


After handling missing values, data shape: (474, 19851)
For the feature 'Red_Hair', the least common label is '0' with 1 occurrences. This represents 0.21% of the dataset.
The distribution of the feature 'Red_Hair' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 70.75
Min: 15.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 180 occurrences. This represents 37.97% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset was determined to be unusable and was not saved.
